# LAMMPS Tutorials 01 nvt. Running your first LAMMPS simulation!

### Author: ABDELLAH TAHIRI 

Please contact me if you have a problem with this tutorial, so I can modify in Github.  I have added FAQs, and will update my versions of LAMMPS in the future to keep the scripts current.

The latest version of this [Jupyter Notebook](http://ipython.org/notebook.html) tutorial is available at https://github.com/athiri78/DM/edit/main/LAMMPS-Tutorials.



***

## Description:
<a id="Sec1"></a>

The objective of this script is to perform an energy minimization of a two-species system in a 3D domain using the Lennard-Jones (LJ) potential. The system consists of two types of particles distributed in cylindrical regions, and the energy is minimized to find a stable configuration.



## Step 1: Download an Input File
<a id="Step1"></a>

This input script was run using the Aug 2024 version of LAMMPS. Changes in some commands may require revision of the input script. To get the input file, you have a few options:

*  Copy the text below and paste it into a text file, `cylindre.in`. Use the `Paste Special` command with unformatted text into the file. 
*  Or, I added the command `%%writefile mixed.in` to the Jupyter Notebook which should just do everything for you!




In [2]:
%%writefile cylindre.in
#######################################################################
# PART A - ENERGY MINIMIZATION
#####################################################################
##########################################################################
# PART A - ENERGY MINIMIZATION
# Script to minimize energy of a two-species system in a 3D cylindrical setup
##########################################################################

# 1) Initialization
units lj                              # Use Lennard-Jones reduced units
dimension 3                           # Set simulation to 3D
atom_style atomic                     # Define atoms as simple particles without internal degrees of freedom
pair_style lj/cut 2.5                 # Lennard-Jones potential with a cutoff distance of 2.5
boundary p p p                        # Apply periodic boundary conditions in all directions

##########################################################################
# 2) System Definition

# Define the simulation box
region simulation_box block -20 20 -20 20 -20 20    # Create a cubic simulation box from -20 to 20 in all directions
create_box 2 simulation_box                         # Initialize the simulation box with two atom types

# Define regions
region region_cylinder_in cylinder z 0 0 10 INF INF side in    # Inner cylinder along the z-axis with radius 10
region region_cylinder_out cylinder z 0 0 10 INF INF side out  # Outer region outside the inner cylinder

# Create atoms
create_atoms 1 random 1000 341341 region_cylinder_out   # Place 1000 atoms of type 1 randomly in the outer cylinder
create_atoms 2 random 150 127569 region_cylinder_in     # Place 150 atoms of type 2 randomly in the inner cylinder

##########################################################################
# 3) Simulation Settings

# Assign masses to atom types
mass 1 1                          # Mass of atom type 1
mass 2 1                          # Mass of atom type 2

# Lennard-Jones interaction parameters
pair_coeff 1 1 1.0 1.0            # ε = 1.0, σ = 1.0 for interactions between atoms of type 1
pair_coeff 2 2 0.5 3.0            # ε = 0.5, σ = 3.0 for interactions between atoms of type 2

##########################################################################
# 4) Visualization and Output Settings

thermo 10                          # Print thermodynamic data every 10 steps
thermo_style custom step temp pe ke etotal press   # Display step, temperature, PE, KE, total energy, and pressure
dump mydmp all atom 10 dump.min.lammpstrj          # Save atom positions to a dump file every 10 steps

##########################################################################
# 5) Energy Minimization

minimize 1.0e-4 1.0e-6 1000 10000   # Minimize energy with tolerances: energy (1.0e-4), force (1.0e-6)
                                    # and limits: 1000 force evaluations, 10,000 iterations
write_data minimized_coordinate.data   # Write minimized atomic configuration to a data file

##########################################################################

print "All done!"

Writing cylindre.in


Awesome!  That little script should have written the above text to the file `nvt.in`.  To check, let's execute a command on the present directory listing all files that end in `*.in`.

In [4]:
!dir *.in

 Le volume dans le lecteur C n’a pas de nom.
 Le numéro de série du volume est DC02-C1A6

 Répertoire de C:\Users\pc\Desktop\M2A

21/11/2024  10:17             1 774 calc_fcc.in
12/11/2024  14:40             1 565 calc_fcc_ver1.in
09/11/2024  14:10             1 233 calc_fcc_ver2.in
23/12/2024  17:51             3 143 cylindre.in
03/12/2024  12:18             1 141 Diatomic.in
03/12/2024  12:22             1 113 Diatomic1.in
12/11/2024  16:49             1 177 lj.in
12/11/2024  17:08               533 ljbarosta.in
12/11/2024  17:19             1 389 ljovito.in
12/11/2024  18:10             1 018 ljrdf.in
12/11/2024  16:59               522 ljthermo.in
25/11/2024  12:17             2 875 minimization.in
25/11/2024  12:45             3 080 minimization1.in
03/12/2024  11:44             1 141 mixed.in
26/11/2024  15:15             3 352 nve.in
26/11/2024  16:50             4 909 nve_npt_nvt.in
23/11/2024  15:26             3 010 nvt.in
              17 fichier(s)           32 975 octets
 

***
## Step 2: Running LAMMPS
### Run this using LAMMPS in Jupyter Notebook
We can actually run this from Jupyter Notebook.  Let's try it.

In [6]:
!lmp -in mixed.in  -pk omp 8 -sf omp 

LAMMPS (29 Aug 2024 - Update 1)
OMP_NUM_THREADS environment is not set. Defaulting to 1 thread. (src/comm.cpp:98)
  using 1 OpenMP thread(s) per MPI task
Loaded 1 plugins from C:\LAMMPS 64-bit 29Aug2024\plugins
set 8 OpenMP thread(s) per MPI task
using multi-threaded neighbor list subroutines
Created orthogonal box = (-20 -20 -20) to (20 20 20)
  1 by 1 by 1 MPI processor grid
Created 1500 atoms
  using lattice units in orthogonal box = (-20 -20 -20) to (20 20 20)
  create_atoms CPU = 0.001 seconds
Created 100 atoms
  using lattice units in orthogonal box = (-20 -20 -20) to (20 20 20)
  create_atoms CPU = 0.000 seconds
Generated 1 of 1 mixed pair_coeff terms from geometric mixing rule
Last active /omp style is pair_style lj/cut/omp
Neighbor list info ...
  update: every = 1 steps, delay = 0 steps, check = yes
  max neighbors/atom: 2000, page size: 100000
  master list distance cutoff = 2.8
  ghost atom cutoff = 2.8
  binsize = 1.4, bins = 29 29 29
  1 neighbor lists, perpetual/occasion

Sweet!

If you want to view the entire file (opening `log.lammps` in Notepad), then:

In [8]:
!type log.lammps

LAMMPS (29 Aug 2024 - Update 1)
OMP_NUM_THREADS environment is not set. Defaulting to 1 thread. (src/comm.cpp:98)
  using 1 OpenMP thread(s) per MPI task
Loaded 1 plugins from C:\LAMMPS 64-bit 29Aug2024\plugins
package omp 8
set 8 OpenMP thread(s) per MPI task
using multi-threaded neighbor list subroutines
#######################################################################
# PART A - ENERGY MINIMIZATION
#####################################################################
# 1) Initialization
units lj
dimension 3
atom_style atomic
pair_style lj/cut 2.5
boundary p p p
######################
# 2) System definition

region simulation_box block -20 20 -20 20 -20 20
create_box 2 simulation_box
Created orthogonal box = (-20 -20 -20) to (20 20 20)
  1 by 1 by 1 MPI processor grid
create_atoms 1 random 1500 341341 simulation_box
Created 1500 atoms
  using lattice units in orthogonal box = (-20 -20 -20) to (20 20 20)
  create_atoms CPU = 0.001 seconds
create_atoms 2 random 100 127569 simulat

In [14]:
!type dump.lam


Le fichier spécifié est introuvable.


***
## Part A - Energy Minimization

* Initialization:

units lj: Defines Lennard-Jones (LJ) units for the simulation.
dimension 3: Specifies a 3D simulation.
atom_style atomic: Defines the atom style as atomic, which is typical for non-bonded systems.
pair_style lj/cut 2.5: Specifies the pair potential (Lennard-Jones) with a cutoff distance of 2.5 units.
boundary p p p: Periodic boundary conditions in all three directions.
* System Definition:

region simulation_box block -20 20 -20 20 -20 20: Defines a cubic region for the simulation box with boundaries from -20 to 20 in each direction.
create_box 2 simulation_box: Creates a simulation box for 2 atom types within the defined region.
create_atoms 1 random 1500 341341 simulation_box: Creates 1500 atoms of type 1 randomly inside the box, with a specified random seed.
create_atoms 2 random 100 127569 simulation_box: Creates 100 atoms of type 2, also randomly, with another random seed.
* Simulation Settings:

mass 1 1: Sets the mass of atom type 1 to 1.
mass 2 1: Sets the mass of atom type 2 to 1.
pair_coeff 1 1 1.0 1.0: Specifies the interaction parameters for atom type 1 with itself (Lennard-Jones parameters: epsilon=1.0, sigma=1.0).
pair_coeff 2 2 0.5 3.0: Specifies the interaction parameters for atom type 2 with itself (epsilon=0.5, sigma=3.0).
* Visualization:

thermo 10: Prints thermodynamic data every 10 steps.
thermo_style custom step temp pe ke etotal press: Customizes the thermodynamic output to show the simulation step, temperature, potential energy, kinetic energy, total energy, and pressure.
* Run:

minimize 1.0e-4 1.0e-6 1000 10000: Runs an energy minimization process to minimize forces and energy with convergence criteria.
Part B - Molecular Dynamics
This part sets up and runs a molecular dynamics simulation after energy minimization.

* Visualization:

thermo 50: Outputs thermodynamic data every 50 steps during the MD run.
* Run:

- fix mynve all nve: Uses the NVE ensemble (constant number of atoms, volume, and energy) to integrate the equations of motion.
- fix mylgv all langevin 1.0 1.0 0.1 1530917: Applies Langevin dynamics for thermostatting, with a damping coefficient of 1.0 and a temperature range - - from 1.0 to 1.0 (constant temperature).
- timestep 0.005: Defines the time step for the MD simulation as 0.005.
- dump mydmp all atom 100 dump.lammpstrj: Dumps atomic positions every 100 steps to a file named dump.lammpstrj for visualization or analysis.
- run 10000: Runs the MD simulation for 10,000 steps.





***
## Exercise
<br>

Learn how to set up, modify, and analyze a simulation for energy minimization of a two-species system distributed in a cylindrical region using the Lennard-Jones potential.


1- Understanding the Script:

Read through the provided LAMMPS script. Pay close attention to the comments to understand the purpose of each section.
Identify the following:
The regions defined in the system.
The properties of the two atom types (e.g., masses, interaction parameters).
2- Modify the Script:

Change the radius of the inner cylindrical region (region_cylinder_in) from 10 to 15.
Increase the number of type 1 atoms created in region_cylinder_out to 2000.
Update the thermo command to print thermodynamic data every 20 steps instead of 10.
Run the Simulation:

3- Use LAMMPS to run the script after making the modifications.
Observe the output files:
Check the thermodynamic output in the console.
Visualize the dump.min.lammpstrj file in a visualization tool (e.g., OVITO or VMD).
3- Analyze the Results:

Find the minimized potential energy (pe) and total energy (etotal) of the system from the thermodynamic output.
Compare the changes in energy with and without your modifications (i.e., before and after increasing the cylinder radius and atom numbers).
4- Questions: Answer the following questions based on your analysis:

a-What effect does increasing the radius of the inner cylinder have on the distribution of type 2 atoms?
b- How does doubling the number of type 1 atoms affect the potential energy of the system?
c- What is the significance of the Lennard-Jones interaction parameters for type 2 atoms (ε = 0.5, σ = 3.0) compared to type 1 atoms?



***
## Tutorial Links

[Click here to open the next tutorial](LAMMPS-Tutorials-02.ipynb)